In [9]:
import polars as pl
import pyarrow.parquet as pq
import requests 
from io import StringIO

def download_csv_to_dataframe(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = StringIO(response.text)
        return pl.read_csv(data)
    else:
        print(f"Failed to download data from {url}. Status code: {response.status_code}")
        return None

url_zhvi = "https://files.zillowstatic.com/research/public_csvs/zhvi/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
url_fips = "https://raw.githubusercontent.com/kjhealy/fips-codes/master/state_and_county_fips_master.csv"
# url_fips_other = "https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv"

df_zhvi = download_csv_to_dataframe(url_zhvi)
df_fips = download_csv_to_dataframe(url_fips)
# df_fips_other = download_csv_to_dataframe(url_fips_other)

print(f"Zillow Data: {df_zhvi.shape}")
print(f"FIPS: {df_fips.shape}")

# create a FIPS column in the Zillow Data  
# join on the FIPS df to create the necessary amount of rows


Zillow Data: (3074, 293)
FIPS: (3195, 3)


In [10]:
df_zhvi = (
    df_zhvi
    .with_columns([
        df_zhvi['StateCodeFIPS'].cast(pl.datatypes.Utf8).alias('StateCodeFIPS_str'),
        df_zhvi['MunicipalCodeFIPS'].cast(pl.datatypes.Utf8).str.zfill(3).alias('MunicipalCodeFIPS_str')
    ])

    .with_columns((pl.col('StateCodeFIPS_str') + pl.col('MunicipalCodeFIPS_str')).alias('CombinedFIPS'))
)

df_zhvi.select(['StateCodeFIPS', 'MunicipalCodeFIPS', 'CombinedFIPS']).head()

StateCodeFIPS,MunicipalCodeFIPS,CombinedFIPS
i64,i64,str
6,37,"""6037"""
17,31,"""17031"""
48,201,"""48201"""
4,13,"""4013"""
6,73,"""6073"""


In [11]:
# drop the StateCodeFIPS and MunicipalCodeFIPS - now that they're combined we don't need them
df_zhvi = df_zhvi.drop(['StateCodeFIPS', 'MunicipalCodeFIPS', 'StateCodeFIPS_str', 'MunicipalCodeFIPS_str', 'RegionName', 'RegionType', 'StateName', 'State'])

# convert the combined FIPS to int
df_zhvi = df_zhvi.with_columns(df_zhvi['CombinedFIPS'].cast(pl.datatypes.Int64).alias('CombinedFIPS'))

# move the combined column to the front
df_zhvi = df_zhvi.select(['CombinedFIPS'] + [col for col in df_zhvi.columns if col != 'CombinedFIPS'])

df_zhvi.head()


CombinedFIPS,RegionID,SizeRank,Metro,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,…,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31
i64,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
6037,3101,0,"""Los Angeles-Lo…",209825.441364,210062.704797,210938.627278,212672.921062,214878.521291,216988.621477,219115.126044,221302.170479,223328.233552,224945.556233,226420.448747,227824.798336,228752.306247,229618.090006,230841.916203,232930.333219,235231.899829,237559.893696,239943.847291,242396.034101,244598.680438,246751.641592,248703.231217,250722.115594,252236.16492,253940.792048,256207.593818,259386.473834,263027.983977,266657.648556,271016.69597,275681.622361,280651.26943,…,647318.365151,659225.75492,672827.413127,687490.291394,701377.20011,709989.292407,714877.707032,716840.207989,722794.701536,731705.602941,742921.298553,754917.899129,764291.492013,771903.177774,778079.068518,785230.850036,793388.267134,802668.617207,813333.361181,825230.228004,836012.42691,844888.021757,850106.7042,852820.168649,851190.167631,847107.730936,842255.210963,838875.102612,836088.110143,827762.464998,816200.59316,805192.654417,801306.800688,803536.507334,809882.629044,820365.016404,833375.844653
17031,139,1,"""Chicago-Naperv…",147919.131948,147889.44458,148133.020864,148833.449694,149729.786777,150665.973562,151530.429593,152753.730544,154177.941954,155893.345192,157841.564164,159749.148413,161325.405601,162280.428808,163197.99729,164206.909994,165359.325313,166462.193788,167375.370975,168618.61095,170066.687673,171889.890327,173692.775376,175248.883135,176556.046355,177523.220618,178607.627582,179546.450214,180529.826058,181333.620822,182341.839671,183759.746479,185541.639838,…,239766.228445,241091.312052,243413.373236,246245.328836,249102.767496,252064.520116,255597.733217,259370.930128,262835.399039,265522.655306,267783.011744,269575.964605,270535.107417,270328.343582,269993.909588,270181.012214,271340.307496,273681.16593,277434.581046,282252.772924,286831.099734,289379.447416,290556.997652,290083.135423,288797.508101,286609.752173,284761.991193,283855.087203,283433.981019,282706.938801,282332.915489,282659.080156,284121.480878,285665.292471,287398.386561,289322.143944,291643.418981
48201,1090,2,"""Houston-The Wo…",109983.672653,109955.435819,109793.884777,109727.45302,109701.462783,109888.917914,110100.460997,110406.062058,110803.951554,111138.791664,111683.38545,112263.203317,112544.530139,112510.265908,112069.282361,111814.717365,111744.592261,111869.700193,112046.584511,112022.116648,112123.306561,112332.955754,112627.186311,112855.308439,112963.392051,113076.586734,113426.639938,113979.757463,114565.416741,115051.848847,115541.356981,116123.521853,116676.941361,…,208642.631698,210223.541992,212271.206938,214861.058234,217454.578186,220081.124614,222635.014058,225521.470104,228619.424256,232206.9221,236123.03136,239516.035985,241908.146398,243469.077308,245398.142234,248064.551244,251224.7278,255589.427574,260767.059395,266795.149922,272420.370241,277454.043925,281732.328654,284725.537889,286142.804914,286319.265915,286006.020479,285571.043

In [12]:
# df_zhvi.write_csv("output_zhvi.csv")
df_zhvi.head()

CombinedFIPS,RegionID,SizeRank,Metro,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,…,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31
i64,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
6037,3101,0,"""Los Angeles-Lo…",209825.441364,210062.704797,210938.627278,212672.921062,214878.521291,216988.621477,219115.126044,221302.170479,223328.233552,224945.556233,226420.448747,227824.798336,228752.306247,229618.090006,230841.916203,232930.333219,235231.899829,237559.893696,239943.847291,242396.034101,244598.680438,246751.641592,248703.231217,250722.115594,252236.16492,253940.792048,256207.593818,259386.473834,263027.983977,266657.648556,271016.69597,275681.622361,280651.26943,…,647318.365151,659225.75492,672827.413127,687490.291394,701377.20011,709989.292407,714877.707032,716840.207989,722794.701536,731705.602941,742921.298553,754917.899129,764291.492013,771903.177774,778079.068518,785230.850036,793388.267134,802668.617207,813333.361181,825230.228004,836012.42691,844888.021757,850106.7042,852820.168649,851190.167631,847107.730936,842255.210963,838875.102612,836088.110143,827762.464998,816200.59316,805192.654417,801306.800688,803536.507334,809882.629044,820365.016404,833375.844653
17031,139,1,"""Chicago-Naperv…",147919.131948,147889.44458,148133.020864,148833.449694,149729.786777,150665.973562,151530.429593,152753.730544,154177.941954,155893.345192,157841.564164,159749.148413,161325.405601,162280.428808,163197.99729,164206.909994,165359.325313,166462.193788,167375.370975,168618.61095,170066.687673,171889.890327,173692.775376,175248.883135,176556.046355,177523.220618,178607.627582,179546.450214,180529.826058,181333.620822,182341.839671,183759.746479,185541.639838,…,239766.228445,241091.312052,243413.373236,246245.328836,249102.767496,252064.520116,255597.733217,259370.930128,262835.399039,265522.655306,267783.011744,269575.964605,270535.107417,270328.343582,269993.909588,270181.012214,271340.307496,273681.16593,277434.581046,282252.772924,286831.099734,289379.447416,290556.997652,290083.135423,288797.508101,286609.752173,284761.991193,283855.087203,283433.981019,282706.938801,282332.915489,282659.080156,284121.480878,285665.292471,287398.386561,289322.143944,291643.418981
48201,1090,2,"""Houston-The Wo…",109983.672653,109955.435819,109793.884777,109727.45302,109701.462783,109888.917914,110100.460997,110406.062058,110803.951554,111138.791664,111683.38545,112263.203317,112544.530139,112510.265908,112069.282361,111814.717365,111744.592261,111869.700193,112046.584511,112022.116648,112123.306561,112332.955754,112627.186311,112855.308439,112963.392051,113076.586734,113426.639938,113979.757463,114565.416741,115051.848847,115541.356981,116123.521853,116676.941361,…,208642.631698,210223.541992,212271.206938,214861.058234,217454.578186,220081.124614,222635.014058,225521.470104,228619.424256,232206.9221,236123.03136,239516.035985,241908.146398,243469.077308,245398.142234,248064.551244,251224.7278,255589.427574,260767.059395,266795.149922,272420.370241,277454.043925,281732.328654,284725.537889,286142.804914,286319.265915,286006.020479,285571.043

In [36]:
zhvi = df_fips.join(df_zhvi, left_on="fips", right_on="CombinedFIPS", how="left")
mask = zhvi["name"].str.to_uppercase() != zhvi["name"]
zhvi = zhvi.filter(mask)
zhvi = zhvi.drop(["state", "RegionID", "name"])
zhvi.head()
# zhvi.write_csv("output_zhvi.csv")
